In [1]:

!pip3 install z3-solver


In [ ]:
import io
import math
import time
import os
from z3 import *

def solve():
  s = Solver()

  # variables
#basing on Order Encoding
#we define 2 boolen matrices posX and posY where x,y are left corner of each board
#

  posX = [[Bool(f"posX_{i+1}_{x}") for x in range(W)] for i in range(blocks)]  
  posY = [[Bool(f"posY_{j+1}_{y}") for y in range(H)] for j in range(blocks)] 

  #we define 2 matrices left & upper with same dimension boards×boards
  #left matrice returns true if board i is on left of board j
  #upper matirce returns True if board j is placed up on i 
  # we assume in both cases i is not eaual to j 

  left = [[Bool(f" left_{i+1}_{j+1}") if i != j else 0 for j in range(blocks)] for i in range(blocks)]
  upper= [[Bool(f" upper_{i+1}_{j+1}") if i != j else 0 for j in range(blocks)] for i in range(blocks)]



  for i in range(blocks): 
    # Variables exceeding the constraints are set to true
    for x in range(W - actual_width[i], W):
      s.add(posX[i][x])
    for y in range(H - actual_height[i], H):
      s.add(posY[i][y])

 # non-overlapping constraints

   # Order encoding constraints
    for x in range(W - actual_width[i]):  
      s.add(Or(Not(posX[i][x]), posX[i][x+1]))
    for y in range(H - actual_height[i]):  
      s.add(Or(Not(posY[i][y]), posY[i][y+1]))

  # 4-literal clauses as non-overlapping constraints
  for i in range(blocks):  
    for j in range(blocks):
      if i >= j: 
        continue

      s.add(Or(left[i][j], left[j][i], upper[i][j], upper[j][i]))

  
  # 3-literal clauses as non-overlapping constraints

  #  functions useful for 3-literal clauses of non-overlapping constraint
  def NoOverlap_encoding_x(i, j):
    res = []
    res.append([Not(posX[j][actual_width[i] - 1])])
    for x in range(W - actual_width[i] - 1):
      res.append([posX[i][x], Not(posX[j][x + actual_width[i]])])

    res.append([posX[i][W - actual_width[i] - 1]])
    return res


  def NoOverlap_encoding_y(i, j):
    res = []
    res.append([Not(posY[j][actual_height[i] - 1])])
    for y in range(H - actual_height[i] - 1):
      res.append([posY[i][y], Not(posY[j][y +actual_height[i]])])

    res.append([posY[i][H - actual_height[i] - 1]])
    return res


  for i in range(blocks):
    for j in range(i+1, blocks):
        
      for c in NoOverlap_encoding_x(i, j):
        prop = [Not(left[i][j])] + c
        s.add(Or(prop))
        
      for c in NoOverlap_encoding_x(j, i):
        prop = [Not(left[j][i])] + c
        s.add(Or(prop))

      for c in NoOverlap_encoding_y(i, j):
        prop = [Not(upper[i][j])] + c
        s.add(Or(prop))
        
      for c in NoOverlap_encoding_y(j, i):
        prop = [Not(upper[j][i])] + c
        s.add(Or(prop))
# given two rectangles, if the sum of the widths (or heights)exceed the maximumum 
# we add 2 constraints in order to avoid that th2 Rectangles be:
# 1-placed side by side:
      if actual_width[i] + actual_width[j] > W:
        s.add(And(Not( left[i][j]), Not(left[j][i])))    
# 2- be placed one over the other:
      if actual_height[i] + actual_height[j] > H:
        s.add(And(Not(upper[i][j]), Not(upper[j][i])))
        
        
        
# Symmetry breaking constraints:
# in case when 2  rectangles of same height and width, the first one is forced to be on the left of the second o
 
  for i in range(blocks):
    for j in range(i+1, blocks):
      
      if actual_width[i] == actual_width[j] and actual_height[i] == actual_height[j]:
        s.add(Not( left[j][i]))
        s.add(Or(left[i][j], Not(upper[j][i])))



  s.set('timeout', 300 * 1000)
  if s.check() == sat:
    m = s.model()
    print('Solved')
  else:
    print('Not solvable')

  return s, posX, posY


# Converter: SAT boolean variables are translated in cartesian coordinates
def converter_sat_coord(m, posX, posY):
  x_sol = []
  y_sol = [] 

  for i in range(blocks):
    j = 0
    while j < W:
      if m.evaluate(posX[i][j]):
        x_sol.append(j)
        break
      j += 1

    j = 0
    while j < H:
      if m.evaluate(posY[i][j]):
        y_sol.append(j)
        break
      j += 1

  return [x_sol, y_sol]


# Reading file


for i in range(1, 41):
  file_name = "./instancesTxt/ins-" + str(i) + ".txt"
  buf = open(file_name)
  W = int(buf.readline())
  blocks = int(buf.readline())

  actual_width = []
  actual_height = []

  for line in buf:
    tmp = line.split()
    tmp = [int(x) for x in tmp]
    actual_width.append(tmp[0])
    actual_height.append(tmp[1])
    
  #Deffinh H Matematically   
  # H is the objective Value that highlight the optimality 
  H = int(math.ceil(sum([actual_width[i] * actual_height[i] for i in range(blocks)]) / W))
  
  start = time.time()
  s, posX, posY = solve()
  end = time.time()

  print(file_name + ' ' + "{:.2f}".format(end - start))

  cornerx, cornery = converter_sat_coord(s.model(), posX, posY)
#Saving output of instances in SAtoutput File

  print(os.getcwd())
  SAToutput_file = '../SAToutput/outputofinstance-{}.txt'.format(i)
  outputofinstance_buf = open(SAToutput_file, 'w')
  outputofinstance_buf.write(str(W) + ' ' + str(H) + '\n')
  outputofinstance_buf.write(str(blocks) + '\n')

  for (w, h, cx, cy) in zip(actual_width, actual_height, cornerx, cornery):
    outputofinstance_buf.write(str(w) + ' ' + str(h) + ' ' + str(cx) + ' ' + str(cy) + '\n')


Solved
./instancesTxt/ins-1.txt 0.07
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-2.txt 0.11
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-3.txt 0.19
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-4.txt 0.25
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-5.txt 0.34
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-6.txt 0.56
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-7.txt 0.62
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-8.txt 0.79
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 (CP + SMT + SAT )\SAT\src
Solved
./instancesTxt/ins-9.txt 0.84
C:\Users\Tony\Desktop\vlsi\previous\VLSI-Project 1 